In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 1
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC10(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC10(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC10(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
讀取3秒資料
讀取5秒資料
Data Loading Time: 0.05 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []


#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    #------------------------------------------------
    #需要調整的地方
    #------------------------------------------------
    #學習率
    lr= 0.01
    #epoch
    epoch= 90
    #訓練資料(要用幾秒訓練) data_1S/data_3S/data_5S
    trainData = data_3S
    #------------------------------------------------
    
    #紀錄訓練開始時間
    start_time = time.time()

    #訓練模型
    model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
    #測試準確度
    test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
    test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
    test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

    #紀錄訓練結束時間
    end_time = time.time()

    #計算訓練耗時
    training_time = end_time - start_time

    #印出訓練耗時
    print("Training Time: {:.2f} seconds".format(training_time))

    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("訓練過程\n")
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    for i in range(len(train_loss)):
        # 寫入資料
        f.write("Fold: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
    f.write("Training Time: {:.2f} seconds\n".format(training_time))
    f.write("---------------------------------------------------------------------------------------------------------------\n")

    #--------------------------------------------------------------------

    
    kFoldTrainTime.append(training_time)
    kFoldTrainAccuracy.append(train_accuracy[-1])
    kFoldTestAccuracy1.append(test_accuracy1)
    kFoldTestAccuracy3.append(test_accuracy3)
    kFoldTestAccuracy5.append(test_accuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
可用的 GPU 数量: 1
Train Epoch: 1/90 Train_Loss: 1.2454969882965088 Train_acc: 30.416668%
Train Epoch: 2/90 Train_Loss: 0.8841696381568909 Train_acc: 65.520836%
Train Epoch: 3/90 Train_Loss: 0.528398871421814 Train_acc: 79.583336%
Train Epoch: 4/90 Train_Loss: 0.3295556306838989 Train_acc: 83.333336%
Train Epoch: 5/90 Train_Loss: 0.44893333315849304 Train_acc: 86.770836%
Train Epoch: 6/90 Train_Loss: 0.17034092545509338 Train_acc: 91.145836%
Train Epoch: 7/90 Train_Loss: 0.13667593896389008 Train_acc: 92.708336%
Train Epoch: 8/90 Train_Loss: 0.10295628011226654 Train_acc: 94.375008%
Train Epoch: 9/90 Train_Loss: 0.10198330134153366 Train_acc: 96.145836%
Train Epoch: 10/90 Train_Loss: 0.07714825123548508 Train_acc: 95.312508%
Train Epoch: 11/90 Train_Loss: 0.12

Train Epoch: 15/90 Train_Loss: 0.09000120311975479 Train_acc: 97.187508%
Train Epoch: 16/90 Train_Loss: 0.11061182618141174 Train_acc: 98.020836%
Train Epoch: 17/90 Train_Loss: 0.1254156082868576 Train_acc: 98.229172%
Train Epoch: 18/90 Train_Loss: 0.07655294984579086 Train_acc: 98.229172%
Train Epoch: 19/90 Train_Loss: 0.050259578973054886 Train_acc: 98.333336%
Train Epoch: 20/90 Train_Loss: 0.060327932238578796 Train_acc: 98.437508%
Train Epoch: 21/90 Train_Loss: 0.06547416001558304 Train_acc: 98.437508%
Train Epoch: 22/90 Train_Loss: 0.039100077003240585 Train_acc: 98.333336%
Train Epoch: 23/90 Train_Loss: 0.04247121140360832 Train_acc: 98.541672%
Train Epoch: 24/90 Train_Loss: 0.06640452891588211 Train_acc: 98.437508%
Train Epoch: 25/90 Train_Loss: 0.031114133074879646 Train_acc: 98.437508%
Train Epoch: 26/90 Train_Loss: 0.04525916278362274 Train_acc: 98.645836%
Train Epoch: 27/90 Train_Loss: 0.06650757789611816 Train_acc: 98.750008%
Train Epoch: 28/90 Train_Loss: 0.035175211727619

Train Epoch: 32/90 Train_Loss: 0.038254883140325546 Train_acc: 98.645836%
Train Epoch: 33/90 Train_Loss: 0.0327601432800293 Train_acc: 98.645836%
Train Epoch: 34/90 Train_Loss: 0.047180261462926865 Train_acc: 98.645836%
Train Epoch: 35/90 Train_Loss: 0.02966444194316864 Train_acc: 98.645836%
Train Epoch: 36/90 Train_Loss: 0.009441554546356201 Train_acc: 98.645836%
Train Epoch: 37/90 Train_Loss: 0.05341080576181412 Train_acc: 98.645836%
Train Epoch: 38/90 Train_Loss: 0.041492678225040436 Train_acc: 98.541672%
Train Epoch: 39/90 Train_Loss: 0.1103443056344986 Train_acc: 98.645836%
Train Epoch: 40/90 Train_Loss: 0.03038470447063446 Train_acc: 98.645836%
Train Epoch: 41/90 Train_Loss: 0.01928030513226986 Train_acc: 98.645836%
Train Epoch: 42/90 Train_Loss: 0.001947578857652843 Train_acc: 98.645836%
Train Epoch: 43/90 Train_Loss: 0.0368163175880909 Train_acc: 98.645836%
Train Epoch: 44/90 Train_Loss: 0.1579139530658722 Train_acc: 98.645836%
Train Epoch: 45/90 Train_Loss: 0.0470847487449646 

Train Epoch: 49/90 Train_Loss: 0.013997121714055538 Train_acc: 98.645836%
Train Epoch: 50/90 Train_Loss: 0.013078068383038044 Train_acc: 98.645836%
Train Epoch: 51/90 Train_Loss: 0.003943395335227251 Train_acc: 98.645836%
Train Epoch: 52/90 Train_Loss: 0.03234492614865303 Train_acc: 98.541672%
Train Epoch: 53/90 Train_Loss: 0.014384817332029343 Train_acc: 98.645836%
Train Epoch: 54/90 Train_Loss: 0.031228989362716675 Train_acc: 98.645836%
Train Epoch: 55/90 Train_Loss: 0.0349070243537426 Train_acc: 98.645836%
Train Epoch: 56/90 Train_Loss: 0.038805168122053146 Train_acc: 98.645836%
Train Epoch: 57/90 Train_Loss: 0.10259674489498138 Train_acc: 98.645836%
Train Epoch: 58/90 Train_Loss: 0.008670900017023087 Train_acc: 98.645836%
Train Epoch: 59/90 Train_Loss: 0.10611806809902191 Train_acc: 98.645836%
Train Epoch: 60/90 Train_Loss: 0.013471120037138462 Train_acc: 98.645836%
Train Epoch: 61/90 Train_Loss: 0.012669115327298641 Train_acc: 98.645836%
Train Epoch: 62/90 Train_Loss: 0.0037097122

Train Epoch: 66/90 Train_Loss: 0.023932768031954765 Train_acc: 98.541672%
Train Epoch: 67/90 Train_Loss: 0.14601199328899384 Train_acc: 98.541672%
Train Epoch: 68/90 Train_Loss: 0.08968165516853333 Train_acc: 98.541672%
Train Epoch: 69/90 Train_Loss: 0.019096042960882187 Train_acc: 98.541672%
Train Epoch: 70/90 Train_Loss: 0.010628028772771358 Train_acc: 98.541672%
Train Epoch: 71/90 Train_Loss: 0.024584567174315453 Train_acc: 98.541672%
Train Epoch: 72/90 Train_Loss: 0.055067937821149826 Train_acc: 98.541672%
Train Epoch: 73/90 Train_Loss: 0.12480226159095764 Train_acc: 98.541672%
Train Epoch: 74/90 Train_Loss: 0.04710349440574646 Train_acc: 98.541672%
Train Epoch: 75/90 Train_Loss: 0.027990490198135376 Train_acc: 98.541672%
Train Epoch: 76/90 Train_Loss: 0.07666599750518799 Train_acc: 98.541672%
Train Epoch: 77/90 Train_Loss: 0.010576877743005753 Train_acc: 98.541672%
Train Epoch: 78/90 Train_Loss: 0.040811631828546524 Train_acc: 98.541672%
Train Epoch: 79/90 Train_Loss: 0.0129906702

In [5]:
#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    # 寫入資料
    f.write("Fold: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, kFoldTrainAccuracy[fold], kFoldTestAccuracy1[fold], kFoldTestAccuracy3[fold], kFoldTestAccuracy5[fold], kFoldTrainTime[fold]))
    f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(max(kFoldTrainAccuracy), max(kFoldTestAccuracy1), max(kFoldTestAccuracy3), max(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(min(kFoldTrainAccuracy), min(kFoldTestAccuracy1), min(kFoldTestAccuracy3), min(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(np.mean(kFoldTrainAccuracy), np.mean(kFoldTestAccuracy1), np.mean(kFoldTestAccuracy3), np.mean(kFoldTestAccuracy5)))

f.close()